# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ': ', end='')
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
import pandas as pd
import numpy as np

In [6]:
document = ET.parse( './data/mondial_database.xml' )

In [7]:
# Find the 10 countries with the lowest infant mortality rates

countries, rates = [], []
for country in document.iter('country'):
    countries.append(country.find('name').text)
    rate = country.find('infant_mortality')
    if rate != None:
        rates.append(float(rate.text))
    else:
        rates.append(np.nan)
df = pd.DataFrame({'country': countries, 'infant_mortality': rates})
df.sort_values('infant_mortality').head(10)

,country,infant_mortality
38,Monaco,1.81
98,Japan,2.13
117,Bermuda,2.48
36,Norway,2.48
106,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


In [8]:
# Find the 10 cities with the largest populations

countries, cities, populations = [], [], []
for country in document.iter('country'):
    for city in country.iter('city'):
        countries.append(country.find('name').text)
        cities.append(city.find('name').text)
        pops = city.findall('population')
        if pops != []:
            populations.append(int(pops[-1].text))
        else:
            populations.append(np.nan)
df = pd.DataFrame({'city': cities, 'country': countries, 'population': populations})
df.sort_values('population', ascending=False).head(10)

,city,country,population
1341,Shanghai,China,22315474
771,Istanbul,Turkey,13710512
1527,Mumbai,India,12442373
479,Moskva,Russia,11979529
1340,Beijing,China,11716620
2810,São Paulo,Brazil,11152344
1342,Tianjin,China,11090314
1064,Guangzhou,China,11071424
1582,Delhi,India,11034555
1067,Shenzhen,China,10358381


In [9]:
# Find the 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

groups = list(set([g.text for g in document.findall('country/ethnicgroup')]))
df = pd.DataFrame({'ethnic_group': groups, 'population': [0 for g in groups]})
for country in document.iter('country'):
        country_pop = float(country.findall('population')[-1].text)
        for group in country.iter('ethnicgroup'):
            group_pop = float(group.attrib['percentage']) / 100 * country_pop
            df.loc[df.ethnic_group == group.text, 'population'] += group_pop
df.sort_values('population', ascending=False).head(10)

,ethnic_group,population
98,Han Chinese,1.245059e+09
251,Indo-Aryan,8.718156e+08
91,European,4.948722e+08
100,African,3.183251e+08
60,Dravidian,3.027137e+08
22,Mestizo,1.577344e+08
54,Bengali,1.467769e+08
181,Russian,1.318570e+08
229,Japanese,1.265342e+08
170,Malay,1.219936e+08


In [10]:
# Find the name and country of the longest river

country_codes = {}
for country in document.iter('country'):
    country_codes[country.attrib['car_code']] = country.find('name').text
    
def decode_countries(codes):
    return ', '.join([country_codes[c] for c in codes.split()])

rivers, countries, lengths = [], [], []
for river in document.iter('river'):
    rivers.append(river.find('name').text)
    countries.append(decode_countries(river.attrib['country']))
    length = river.find('length')
    if length != None:
        lengths.append(float(length.text))
    else:
        lengths.append(np.nan)
df = pd.DataFrame({'river': rivers, 'country': countries, 'length': lengths})
df.sort_values('length', ascending=False).head()

,country,length,river
174,"Colombia, Brazil, Peru",6448,Amazonas
137,China,6380,Jangtse
136,China,4845,Hwangho
123,Russia,4400,Lena
205,"Congo, Zaire",4374,Zaire


In [11]:
# Find the name and country of the largest lake

lakes, countries, areas = [], [], []
for lake in document.iter('lake'):
    lakes.append(lake.find('name').text)
    countries.append(decode_countries(lake.attrib['country']))
    area = lake.find('area')
    if area != None:
        areas.append(float(area.text))
    else:
        areas.append(np.nan)
df = pd.DataFrame({'lake': lakes, 'country': countries, 'area': areas})
df.sort_values('area', ascending=False).head()

,area,country,lake
54,386400,"Russia, Azerbaijan, Kazakhstan, Iran, Turkmeni...",Caspian Sea
109,82103,"Canada, United States",Lake Superior
81,68870,"Tanzania, Kenya, Uganda",Lake Victoria
106,59600,"Canada, United States",Lake Huron
108,57800,United States,Lake Michigan


In [12]:
# Find the name and country of the airport at highest elevation

airports, countries, elevations = [], [], []
for airport in document.iter('airport'):
    airports.append(airport.find('name').text)
    countries.append(decode_countries(airport.attrib['country']))
    elevation = airport.find('elevation')
    if elevation != None and elevation.text != None:
        elevations.append(float(elevation.text))
    else:
        elevations.append(np.nan)
df = pd.DataFrame({'airport': airports, 'country': countries, 'elevation': elevations})
df.sort_values('elevation', ascending=False).head()

,airport,country,elevation
80,El Alto Intl,Bolivia,4063
219,Lhasa-Gonggar,China,4005
241,Yushu Batang,China,3963
813,Juliaca,Peru,3827
815,Teniente Alejandro Velasco Astete Intl,Peru,3311
